In [ ]:
import os
from diffaux.disk_bulge_modeling.generate_bulge_disk_sample import (
    get_bulge_disk_test_sample,
    get_bulge_disk_decomposition,
    get_zindexes
)
from jax import random as jran
import numpy as np
import jax.numpy as jnp

ran_key = jran.key(0)

In [ ]:
halo_key, ran_key = jran.split(ran_key, 2)
lgmp_min = 11.0
redshift = 0.05
Lbox = 100.0
diffstar_cens = get_bulge_disk_test_sample(halo_key, lgmp_min=lgmp_min, redshift=redshift, Lbox=Lbox)

## logsm0 Bug-fixed Version

In [ ]:
disk_bulge_key, ran_key = jran.split(ran_key, 2)
diffstar_cens = get_bulge_disk_decomposition(disk_bulge_key, diffstar_cens, new_model=False)
print(diffstar_cens.keys())

In [ ]:
dz = 0.1
zvalues = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
redshifts = diffstar_cens["z_table"]
zindexes, zs = get_zindexes(zvalues, redshifts)
print(zs, zindexes)

In [ ]:
from diffaux.validation.plot_disk_bulge import (
    plot_qs_nocuts,
    plot_histories,
    plot_q_with_cuts,
    plot_q1_q2,
    plot_q1_vs_q2,
    plot_q_vs_qx_at_z_profile,
)

plotdir = "/Users/kovacs/cosmology/BulgeDisk/DiskBulgePlots"

In [ ]:
logMz0_min = 8.0
logssfr_min = -15
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
sfr_mask = np.log10(diffstar_cens["sSFR"][:, -1]) > logssfr_min
mask = mass_mask & sfr_mask
title = "$M^*_{{z=0}} > 10^{{{:.1f}}} M_\\odot, sSFR > 10^{{{}}} yr^{{-1}}$".format(logMz0_min, logssfr_min)
xname = "log_M0_min_{:.1f}_logssfr_min_{}".format(logMz0_min, -logssfr_min)

In [ ]:
plot_q1_vs_q2(
    jnp.log10(diffstar_cens["sSFR"][mask]),
    diffstar_cens["bth"][mask],
    zvalues,
    redshifts,
    jnp.log10(diffstar_cens["smh"][mask]),
    title=title,
    xname=xname,
    cbar_title="$\\log_{10}(M^*/M_\\odot)$",
    N=3000,
    xmin=-14,
    xmax=-8.4,
    ymin=0,
    ymax=1.0,
    ylabel="B/T",
    xlabel="$\\log_{10}(sSFR/yr)$",
    pltname="BoverT_vs_sSFR_{}.png",
    plotdir=plotdir,
)

In [ ]:
logMz0_min = 8.0
logssfr_min = -15
logM_min = 7.0
mass_mask = np.log10(diffstar_cens["smh"][:, -1]) > logMz0_min
sfr_mask = np.log10(diffstar_cens["sSFR"][:, -1]) > logssfr_min
mask = mass_mask & sfr_mask
title = "$M^*_{{z=0}} > 10^{{{:.1f}}} M_\\odot, sSFR > 10^{{{}}} yr^{{-1}}$".format(logMz0_min, logssfr_min)
xname = "log_M0_min_{:.1f}_logssfr_min_{}".format(logMz0_min, -logssfr_min)

In [ ]:
plot_q1_vs_q2(
    jnp.log10(diffstar_cens["smh"][mask]),
    diffstar_cens["bth"][mask],
    zvalues,
    redshifts,
    jnp.log10(diffstar_cens["sSFR"][mask]),
    title=title,
    xname=xname, cmap="jet_r",
    N=2000,
    ymin=0,
    ymax=1.0,
    xmin=7,
    xmax=12,
    xlabel="$\\log_{10}(M^*/M_\\odot)$",
    ylabel="B/T", cbar_title="$\\log_{10}(sSFR/yr)$",
    pltname="BoverT_vs_logMstar_{}.png",
    plotdir=plotdir,
)

In [ ]:
plot_q1_vs_q2(
    diffstar_cens["eff_bulge"][mask],
    diffstar_cens["bth"][mask],
    zvalues,
    redshifts,
    jnp.log10(diffstar_cens["smh"][mask]),
    title=title,
    xname=xname, cmap="jet",
    N=2000,
    ymin=0,
    ymax=1.0,
    xmin=0,
    xmax=1.,
    xlabel="$\\epsilon_{bulge}$",
    ylabel="B/T", cbar_title="$\\log_{10}(M^*/M_\\odot)$",
    pltname="BoverT_vs_eff_bulge_{}.png",
    plotdir=plotdir,
)

In [ ]:
# profiles
qs = [diffstar_cens["bth"][mask], diffstar_cens["eff_bulge"][mask]] 
ylabels = ['B/T', '\\epsilon_{bulge}']
logM_min = 7.0
logM_max = 11.5
Nm = 9
Mbins = np.linspace(logM_min, logM_max, Nm+1)
logsSFR_min = -16
logsSFR_max = -8
Ns = 8
sbins = np.linspace(logsSFR_min, logsSFR_max, Ns +1)

In [ ]:
plot_q_vs_qx_at_z_profile(qs, jnp.log10(diffstar_cens['smh'][mask]), ylabels, 
              zindexes, zs, bins=Mbins,
              title=title,
              xname=xname,
              plotdir=plotdir,
              pltname="BoverT_vs_log_Mz_{}.png",
              errors=[True, True],
              qs_depends_z=True,
)

In [ ]:
plot_q_vs_qx_at_z_profile(qs, jnp.log10(diffstar_cens['sSFR'])[mask], ylabels, 
              zindexes, zs, bins=sbins,
              title=title, 
              xname=xname, xlabel='$\\log_{10}(sSFR*yr)$',
              plotdir=plotdir,
              pltname="BoverT_vs_log_sSFR_{}.png",
              errors=[True, True],
              qs_depends_z=True,
)